# Aujourd'hui on roule sur les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html

# Imports des librairies de bases

On ajoutera celles qui manquent au fur et à mesure de nos besoins

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os, gc

# Définition de la seed pour le random

Très important pour qu'on voit les mêmes choses entre nos deux ordis

In [2]:
RANDOM_SEED = 42;
np.random.seed(RANDOM_SEED)

# Définition des paramètres pour Matplot

Rien de bien intéréssant

In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Set des variables globales

Attention, je n'utilise les variables globales pour la gestion des fichiers. Sinon, c'est mort

In [4]:
# Where to save the figures
PROJECT_ROOT_DIR = "..\.."
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "data_processed")

# Fonction pour load les libraires

En vrai, on a juste besoin de pd.read_csv, mais c'était pour faire joli

In [5]:
def load_data(file,data_path=DATA_PROCESSED, sep=';'):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep=';')

# On load les jeux de données

In [6]:
TX_data = load_data(file = "train.csv");
TEST_DATA = load_data(file = "test.csv");

In [7]:
RESULTS = pd.DataFrame({'ID' : []})
RESULTS["ID"]=TEST_DATA["ID"]

In [8]:
RESULTS.head()

,ID
0,100000
1,100001
2,100002
3,100003
4,100004


In [9]:
TEST_DATA.drop("ID", axis=1, inplace=True)

In [10]:
TX_data.drop(['CARD_PAYMENT','COUPON_PAYMENT','RSP_PAYMENT','WALLET_PAYMENT'], axis = 1, inplace = True)
TEST_DATA.drop(['CARD_PAYMENT','COUPON_PAYMENT','RSP_PAYMENT','WALLET_PAYMENT'], axis = 1, inplace = True)

# Jointure entre les X et Y

In [11]:
def datapreprocess(data):
    data=data.apply(pd.to_numeric, errors='ignore')
    
    # Y and X
    try :
        Y=data["CLAIM_TYPE"]
        X=data.drop("CLAIM_TYPE", axis=1,inplace=False)
    except:
        Y=0
        X=data
    # Exclude Objets
    X=X.select_dtypes(exclude=['object'])
    
    # Work on fare
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    X=pd.DataFrame(imp.fit_transform(X),columns=X.columns.values)
 
    return X, Y

In [12]:
X_train, Y_train = datapreprocess(TX_data)
TEST_DATA, _ = datapreprocess(TEST_DATA)

#del TX_data;
gc.collect()

35

# MODEL!

## Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

### Random Forest Solo

In [14]:
params_RF={
            'bootstrap':True, 
            'class_weight':'balanced',
            'criterion':'gini', 
            'max_depth':5, 
            'max_features':'auto',
            'max_leaf_nodes':20, 
            'min_impurity_decrease':0.0,
            'min_impurity_split':None, 
            'min_samples_leaf':10,
            'min_samples_split':2, 
            'min_weight_fraction_leaf':0.0,
            'n_estimators':2000, 
            'n_jobs':-1, 
            'oob_score':False, 
            'random_state':RANDOM_SEED,
            'verbose':0, 
            'warm_start':False
}

In [15]:
rnd_clf = RandomForestClassifier(**params_RF)

In [16]:
rnd_clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=20, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2000, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [17]:
y_pred_rf = rnd_clf.predict(TEST_DATA)

In [18]:
RESULTS["CLAIM_TYPE"] = pd.DataFrame(y_pred_rf)

In [19]:
RESULTS.head()

,ID,CLAIM_TYPE
0,100000,WITHDRAWAL
1,100001,WITHDRAWAL
2,100002,SELLER_CANCEL_POSTERIORI
3,100003,-
4,100004,UNDEFINED


In [20]:
filename = DATA_PROCESSED+"/submission_RF_1.csv"

RESULTS.to_csv(filename, index=False, sep=";")